In [1]:
import os
import pickle
import pandas as pd
from datasets import load_dataset
from huggingface_hub import login

## Список языков, отобранных для датасета:
1. Indonesian
2. French
3. Spanish
4. Japanese
5. Dutch
6. Lithuanian
7. Portuguese
8. Romanian
9. English 

In [2]:
LANGUAGES = ['Indonesian', 'French', 'Spanish', 'Japanese', 'Dutch', 'Lithuanian', 'Portuguese', 'Romanian', 'English']

In [3]:
LANGUAGES_CODES = {
    'Indonesian': 'id',
    'French': 'fr',
    'Spanish': 'es',
    'Japanese': 'ja',
    'Dutch': 'nl',
    'Lithuanian': 'lt',
    'Portuguese': 'pt',
    'Romanian': 'ro',
    'English': 'en'
}

In [4]:
NUMBER_OF_PIECES = 1500
DURATION_SECONDS = 3

In [5]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret('HF_TOKEN')
login(token=HF_TOKEN)

In [6]:
def process_language(language_code, language):
    print(f'processing language: {language} ({language_code})')
    try:
        dataset = load_dataset(
            'mozilla-foundation/common_voice_16_1', 
            language_code, 
            split='train', 
            streaming=True, 
            trust_remote_code=True
        )
    except Exception as e:
        print(f'failed to load {language}: {e}')
        return []

    language_data = []
    language_count = 0

    for sample in dataset:
        if language_count >= NUMBER_OF_PIECES:
            break

        array = sample['audio']['array']
        array = array.astype('float32')
        sampling_rate = sample['audio']['sampling_rate']
        piece_size = DURATION_SECONDS * sampling_rate
        if len(array) < (sampling_rate * DURATION_SECONDS):
            continue

        number_of_pieces_from_sample = len(array) // piece_size

        for i in range(number_of_pieces_from_sample):
            piece = array[i*piece_size : (i+1)*piece_size]
            
            language_data.append({
                'name': f'{language_code}{language_count}speech',
                'array': piece,
                'language_code': language_code,
                'label': 'speech',
            })
            language_count += 1
    
    print(f'processed {language_count} pieces for language {language} ({language_code})')
    return language_data

In [7]:
for language in LANGUAGES:
    language_data = process_language(LANGUAGES_CODES[language], language)
    if 'speech_dataset.pkl' in os.listdir('/kaggle/working/'):
        with open('speech_dataset.pkl', 'rb') as file:
            existing_data = pd.read_pickle(file)
            language_data = existing_data + language_data
    with open('speech_dataset.pkl', 'wb') as file:
        pickle.dump(language_data, file)

processing language: Indonesian (id)


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

common_voice_16_1.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

languages.py:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

release_stats.py:   0%|          | 0.00/77.3k [00:00<?, ?B/s]

...

processed 1500 pieces for language Portuguese (pt)
processing language: Romanian (ro)


Reading metadata...: 5113it [00:00, 10897.30it/s]


processed 1500 pieces for language Romanian (ro)
processing language: English (en)


Reading metadata...: 1090061it [00:46, 23677.01it/s]


processed 1500 pieces for language English (en)
